## Topic Modeling - Nichtwissenschaft-Datensatz

In [ ]:
!pip install bertopic

In [ ]:
from bertopic import BERTopic

import pandas as pd
import re

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Daten einlesen & Preprocessing

In [ ]:
df = pd.read_excel('tweets_nichtwissenschaft.xlsx')

In [ ]:
df['tweet_text'] = df['tweet_text'].astype(str)
df['retweet_text_retweeted_tweet'] = df['retweet_text_retweeted_tweet'].astype(str)
df['zitat_text'] = df['zitat_text'].astype(str)

In [ ]:
def classify_tweet(row):
    if row['tweet_text'] != '-998' and row['retweet_text_retweeted_tweet'] == '-998' and row['zitat_text'] == '-998':
        return 'tweet'
    elif row['retweet_text_retweeted_tweet'] != '-998':
        return 'retweet'
    elif row['tweet_text'] != '-998' and row['zitat_text'] != '-998':
        return 'zitat_tweet'
    return 'unclassified'

In [ ]:
df['tweet_class'] = df.apply(classify_tweet, axis=1)

In [ ]:
def gather_texts(row):
    texts = []
    for column in ['tweet_text', 'retweet_text_retweeted_tweet', 'zitat_text']:
      if pd.notna(row[column]) and str(row[column]) != '-998':
            texts.append(row[column])
    return ' '.join(texts)

In [ ]:
df['all_tweets'] = df.apply(gather_texts, axis=1)

In [ ]:
def preprocess_tweet(text):
    text = re.sub(r'@\w+', '', text)  # Entfernt Benutzernamen
    text = re.sub(r'http\S+', '', text)  # Entfernt URLs
    text = re.sub(r'www.\S+', '', text)  # Entfernt URLs
    text = re.sub(r'[0-9]+', '', text)  # Entfernt Zahlen
    text = re.sub(r'[^\w\s]', '', text)  # Entfernt Sonderzeichen
    text = re.sub(r'_', '', text)  # Entfernt Unterstriche
    text = re.sub(r'\s+', ' ', text).strip()  # Ersetzt mehrere Leerzeichen durch ein einzelnes und entfernt sie am Anfang/Ende
    text = re.sub(r'-998', '', text)  # entfernt fehlende Werte
    text = re.sub(r'nan', '', text)  # entfernt fehlende Werte

    return text

In [ ]:
df['all_tweets'] = df['all_tweets'].apply(preprocess_tweet)

### Topic Modeling mit BERTopic

In [ ]:
docs = list(df.all_tweets.values)

german_stopwords = set(stopwords.words('german'))
stopwords = list(german_stopwords)
vectorizer_model = CountVectorizer(stop_words=stopwords)

#### Erstes Modell ohne Beschränkung der Themenanzahl

In [ ]:
# BERTopic-Modell erstellen
model = BERTopic(language="German", calculate_probabilities=True, verbose=True, vectorizer_model=vectorizer_model)

# Model anwenden und Topics extrahieren
topics, probs = model.fit_transform(docs)

In [ ]:
# Ergebnisse
print(f"Anzahl der Topics: {model.get_topic_info().shape[0] - 1}")  # Minus 1 wegen des Outliers-Topics (-1)
topic_info = model.get_topic_info()
topic_info.head(20)

Anzahl der Topics: 533


,Topic,Count,Name,Representation,Representative_Docs
0,-1,5265,-1_ja_gibt_forschung_wissenschaft,"[ja, gibt, forschung, wissenschaft, schon, wis...",[So Leute nicht vergessen HEUTE von Uhr MEAwar...
1,0,512,0_malone_löschungen_derailing_sonntagskrimi,"[malone, löschungen, derailing, sonntagskrimi,...",[Sonntagskrimi Legen Sie ihn weg das hier ist ...
2,1,269,1_halbiert_pariser_eeausbau_abkommen,"[halbiert, pariser, eeausbau, abkommen, coemis...",[Manche Menschen verstehen einfach nicht was d...
3,2,251,2_laborthese_bat_clique_farrar,"[laborthese, bat, clique, farrar, unterdrücken...",[Die UNO bat zwei Professoren die Herkunft von...
4,3,219,3_enthüllungen_bertelsmannzensoren_bestätigte_...,"[enthüllungen, bertelsmannzensoren, bestätigte...",[Pfizer bestätigte inzwischen die Enthüllungen...
5,4,175,4_impfung_impfschäden_impfstoff_impfnebenwirku...,"[impfung, impfschäden, impfstoff, impfnebenwir...",[hier nur Auszüge Bevor aber wild spekuliert w...
6,5,171,5_heißt_endemie_respiratorischen_erreger,"[heißt, endemie, respiratorischen, erreger, en...",[Endemie heißt nicht dass ein Erreger harmlos ...
7,6,157,6_ruhiger_therapien_framen_gegenrede,"[ruhiger, therapien, framen, gegenrede, umgede...",[Man kann Gegenrede wenn das Krankheitserleben...
8,7,150,7_directed_reagiert_begrenzten_fällen,"[directed, reagiert, begrenzten, fällen, anzah...",[Pfizer reagiert auf Projekt Veritas Pfizer ha...
9,8,150,8_gegenwärtige_ergebnissen_gründergeneration_m...,"[gegenwärtige, ergebnissen, gründergeneration,...",[Sprecherin F Mayer erzählt dass ihr Verein se...


In [ ]:
model.visualize_topics()

Output hidden; open in https://colab.research.google.com to view.

#### Reduzierung der Themenanzahl

In [ ]:
model.reduce_topics(docs, nr_topics=30)

2024-04-29 12:55:37,970 - BERTopic - Topic reduction - Reducing number of topics
2024-04-29 12:55:40,526 - BERTopic - Topic reduction - Reduced number of topics from 534 to 30


In [ ]:
# Ergebnisse
print(f"Anzahl der Topics: {model.get_topic_info().shape[0] - 1}")  # Minus 1 wegen des Outliers-Topics (-1)
topic_info = model.get_topic_info()
topic_info

Anzahl der Topics: 29


,Topic,Count,Name,Representation,Representative_Docs
0,-1,5265,-1_forschung_gibt_mehr_ja,"[forschung, gibt, mehr, ja, wissenschaft, scho...",[Vor allen Dingen sagt die Forschung schon sei...
1,0,6365,0_forschung_geht_china_klima,"[forschung, geht, china, klima, müssen, mehr, ...",[Manche Menschen verstehen einfach nicht was d...
2,1,5226,1_mecfs_longcovid_forschung_mecfsimbundestag,"[mecfs, longcovid, forschung, mecfsimbundestag...",[Ganz wichtig Am Donnerstag Januar demonstrier...
3,2,1232,2_pfizer_enthüllungen_pfizers_malone,"[pfizer, enthüllungen, pfizers, malone, löschu...",[Sonntagskrimi Legen Sie ihn weg das hier ist ...
4,3,872,3_wieler_lothar_dankewieler_widmen,"[wieler, lothar, dankewieler, widmen, amt, auf...",[Professor Lothar H Wieler verlässt das RKI Er...
5,4,574,4_berlin_protest_nichtgenesen_está,"[berlin, protest, nichtgenesen, está, quem, sy...",[Berlin Heute MECFSimBundestag Stiller Protest...
6,5,358,5_cannabis_zahl_gesamtsterblichkeit_verhinderten,"[cannabis, zahl, gesamtsterblichkeit, verhinde...",[Früher fragte die Forschung ob ein Krebsmitte...
7,6,312,6_frauen_selbstbestimmungsgesetz_gender_transf...,"[frauen, selbstbestimmungsgesetz, gender, tran...",[CN Transfeindlichkeit Wenn ein Institut für G...
8,7,214,7_wer_demokratie_lobbyismus_eskalationen,"[wer, demokratie, lobbyismus, eskalationen, re...",[Den Kern des konservativen Denkens ist eine A...
9,8,209,8_ufoforschung_atlantikzirkulation_kipppunkt_s...,"[ufoforschung, atlantikzirkulation, kipppunkt,...",[Der Kipppunkt der Atlantikzirkulation regt se...


In [ ]:
for i in range(1, 11):
    representation = topic_info.loc[i, 'Representation']
    representative_docs = topic_info.loc[i, 'Representative_Docs']
    print(f"Thema {i}:")
    print(f"Representation: {representation}")
    print(f"Representative_Docs: {representative_docs}")
    print()

Thema 1:
Representation: ['forschung', 'geht', 'china', 'klima', 'müssen', 'mehr', 'gebraucht', 'klimaschützer', 'lieber', 'demonstriert']
Representative_Docs: ['Manche Menschen verstehen einfach nicht was die Klimaforschung sagt in Jahren müssen die COEmissionen halbiert sein um das Pariser Abkommen einzuhalten Also nicht studieren und forschen sondern Ausstieg aus fossilen und EEAusbau sofort sind notwendig Liebe Klimaschützer Euer Ansinnen in allen Ehren Also geht nicht demonstrieren sondern fleissig an die Uni und danach engagiert in die Forschung Falls Ihr lieber demonstriert geht nach China Da werden Demos für das Klima sehr dringend gebraucht', 'Manche Menschen verstehen einfach nicht was die Klimaforschung sagt in Jahren müssen die COEmissionen halbiert sein um das Pariser Abkommen einzuhalten Also nicht studieren und forschen sondern Ausstieg aus fossilen und EEAusbau sofort sind notwendig Liebe Klimaschützer Euer Ansinnen in allen Ehren Also geht nicht demonstrieren sondern f

In [ ]:
len(docs)

21965

In [ ]:
model.visualize_barchart(top_n_topics = 10, n_words = 5)

In [ ]:
model.visualize_topics()

In [ ]:
model.visualize_heatmap()

#### Themen nach Tweet-Kategorie

In [ ]:
topics_per_class = model.topics_per_class(docs,
    classes=df.tweet_class)

3it [00:01,  1.82it/s]


In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 100)


In [ ]:
topics_per_class

,Topic,Words,Frequency,Class,Name
0,-1,"forschung, ja, gibt, mehr, wissenschaft",2877,tweet,-1_forschung_gibt_mehr_ja
1,0,"forschung, geht, mehr, china, ja",2245,tweet,0_forschung_geht_china_klima
2,1,"forschung, mecfs, longcovid, covid, versorgung",1014,tweet,1_mecfs_longcovid_forschung_mecfsimbunde...
3,2,"pfizer, enthüllungen, gain, pfizers, malone",49,tweet,2_pfizer_enthüllungen_pfizers_malone
4,3,"wieler, lothar, widmen, lehre, dankewieler",78,tweet,3_wieler_lothar_dankewieler_widmen
5,4,"archäologie, nachrichten, kultur, meldungen, geschichte",190,tweet,4_berlin_protest_nichtgenesen_está
6,5,"cannabis, krebs, krebsforschung, zahl, krebstoten",75,tweet,5_cannabis_zahl_gesamtsterblichkeit_verh...
7,6,"frauen, trans, gender, forschung, selbstbestimmungsgesetz",87,tweet,6_frauen_selbstbestimmungsgesetz_gender_...
8,7,"demokratie, wer, rechte, rechtsextreme, keingeldfürdieafd",32,tweet,7_wer_demokratie_lobbyismus_eskalationen
9,8,"ufoforschung, update, ifouniversität, ballons, arktis",99,tweet,8_ufoforschung_atlantikzirkulation_kippp...


In [ ]:
model.visualize_topics_per_class(topics_per_class,
    top_n_topics=10, normalize_frequency = True)